<a href="https://colab.research.google.com/github/yfan393/CSE6740/blob/main/cse6040_fa24_09_24_lcs_DRAFT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Problem-solving techniques: slicing, data parallelism, preconditioning, and the "longest consecutive subsequences" problem #

## Link: `bit.ly/4eDM9t7`

_(September 24, 2024)_

* Review: Properties of slices
* Pro-tip: Pretty-printing nested data structures
* Review: Basics of problem solving — outputs, inputs, and strategy
* New topic: "Data parallelism" — elevating your view of problems
* Example: Longest consecutive subsequences (from Codewars)

# Review: Facts about slicing #

Take a moment to review how slicing works. We'll use slices in a subsequent example.

Here is one example: suppose you have the list `L` below and you want to extract the last three elements. Come up with **two slice-based methods** to do it.

In [ ]:
#.   0. 1. 2. 3. 4
L = ["a", "b", "c", "d", "e"]

### YOUR CODE HERE ###

When you reference a list using an invalid index, you get an error. Uncomment the following line to see an example:

In [ ]:
#L[len(L)] # Fails: Why?

Slices work a little differently. Consider the following slice of a list:

In [ ]:
### INSERT: SLICE FROM `len(L)` ###
L[len(L):]

Instead of failing, `L[len(L):]` returns an empty list!

You can think of it this way. First, Python interprets the slice; a range with invalid indices yields an empty range. Referencing the list on an empty range returns an empty list.

Why is this default behavior helpful? You can use it to construct more "robust" algorithms by not having to worry about corner cases, e.g., an empty subset or out-of-bounds ranges.

Here is another example:

In [ ]:
L[-6:1_000_000_000]

# Pro-tip: Pretty-printing nested data structures #

When debugging, consider [`pprint.pprint()`](https://docs.python.org/3/library/pprint.html) to "pretty print" a native-Python nested data structure.

**Example:** Consider the following "list of dictionaries."

In [ ]:
data = [{'first': 'Guido', 'last': 'Van Rossum', 'year': 1956},
        {'first': 'Grace', 'last': 'Hopper',     'year': 1906},
        {'first': 'Alan',  'last': 'Turing',     'year': 1912}]
print(data)

In [ ]:
from pprint import pprint
pprint(data)

# Exercise: Longest consecutive subsequences and "data parallelism" #

Given a string of characters, write some code to find longest consecutive substring of repeated characters. It should then return the character and the number of times it occurred in that longest substring. If there are multiple substrings of repeated characters having the same length, then the function should return the first one that occurs.

In [ ]:
# Examples:
tests = {
    'aaaabb': ('a', 4),
    'bbbaaabaaaa': ('a', 4),
    'bbbaaaabaaa': ('a', 4),
    'bbbaabbaaa': ('b', 3), # tie: return first
    'cbdeuuu900': ('u', 3),
    'abbbbb': ('b', 5),
    'aabb': ('a', 2),
    'ba': ('b', 1),
    '': ('', 0)
}

Below, we'll implement a few variants. To check them, we'll use the following function.

> It's a _higher-order function!_ It takes an implementation of LCS as input and checks it against the test cases shown above.

In [ ]:
def check(implementation):
    for input_string, true_answer in tests.items():
        print(f'* Checking {repr(input_string)}: True solution is {repr(true_answer)}...')
        your_answer = implementation(input_string)
        assert your_answer == true_answer, f'Your code produced {repr(your_answer)} instead.'
    print("\n(Passed!)")

**Exercise.** Implement a program to solve the LCS problem. As a suggestion, start by identifying the **goal** (outputs) and the **inputs**, and then outline a **strategy** (sequence of steps _without_ code) before writing the code.

In [ ]:
def longest_consecutive_subsequence(s):
    # Input: string `s`
    #
    # Output: ('x', m)
    pass

### Baseline method: One-at-a-time ###

Here is a version you might naturally implement. It works as follows.

1. Let `best` and `best_count` hold the letter with the largest count seen so far.
2. Visit each character `c` of the input string `s` from left to right.
3. Determine whether `c` is the same as the previous letter, `previous`, which has occurred `previous_count` times.
4. If `c` is the same as `previous`, increment `previous_count`.
5. Otherwise, see if we can update `best` with `previous`.

In [ ]:
def longest_consecutive_subsequence_0a(s):
    best, best_count = '', 0 # `best` is a single letter, `best_count` is freq in subseq
    previous, previous_count = '', 0 # `previous` will hold a single letter, `previous_count` ...
    for c in s:
        if c == previous:
            previous_count += 1
        else: # c != previous
            if previous_count > best_count:
                best, best_count = previous, previous_count
            previous, previous_count = c, 1
    if previous_count > best_count:
        best, best_count = previous, previous_count
    return best, best_count

First, let's check this solution:

In [ ]:
check(longest_consecutive_subsequence_0a)

The solution shown above has some repetition: the code to update `best` and `best_count` when a new and longer subsequence is discovered appears in two places (lines 8-9 and again in lines 11-12). We'll address that momentarily.

Here is a variation on the above with a few revisions for clarity:

1. _Refactor_ the redundant code into a separate function, `update`. Doing so improves reading, debugging, and maintaining the function. (If you wanted to solve the _shortest_ consecutive subsequence problem instead, you need only change `update`!)
2. Maintain substrings and use `len` to get the length when needed.
3. Use slices to handle empty strings more robustly.

In [ ]:
def longest_consecutive_subsequence_0b(s):
    def update(best, previous):
        return max(best, previous, key=len) # ???

    previous = ''
    best = ''
    for c in s:
        if c not in previous:
            best = update(best, previous)
            previous = ''
        previous += c
    best = update(best, previous)
    return best[-1:], len(best)

check(longest_consecutive_subsequence_0b)

### Method 1: Data parallelism ###

Instead of thinking about building the solution one character at a time, let's try to think about the input "as a whole." What are you really looking for?

In the case of this problem, the input string is really a collection of substrings having the same letter. You can think of finding the solution in two parts: (1) finding the substrings, and then (2) finding the largest one.

For example, consider the input string again:

In [ ]:
s = 'bbbaaabaaaa'

The pieces correspond to these slices of `s`:

In [ ]:
s[0:3], s[3:6], s[6:7], s[7:11]

To reconstruct these substrings, all you need are the locations of _changes_, which occur at positions `[0, 3, 6, 7, 11]`.

To find these, we need to look at consecutive pairs of letters in `s` and see where they differ. Let's write some code to build up this information in two parts.

In [ ]:
### DEMO ###

In [ ]:
def zip_neighbors(s):
    ### YOUR CODE HERE ###
    pass

print("s =", s)
list(zip_neighbors(s))

To associate these neighbor-pairs with positions, apply `enumerate`:

In [ ]:
print("s =", s)

### INSERT: APPLY ENUMERATE ###

These are the building blocks for a "change detector:"

In [ ]:
def detect_changes(s):
    ### INSERT SOLUTION ###
    pass

detect_changes(s)

> _Aside:_ The implementation of `detect_changes` produces the following result on an empty input:

In [ ]:
detect_changes('')

From the changes, the pieces follow:

In [ ]:
def get_pieces(s, changes):
    ### INSERT SOLUTION ###
    pass

get_pieces(s, detect_changes(s))

The largest piece is then easy to find:

In [ ]:
### FIND LARGEST PIECE VIA KEYED `max` ###

Putting it all together:

In [ ]:
def longest_consecutive_subsequence_1(s):
    changes = detect_changes(s)
    pieces = get_pieces(s, changes)
    largest_piece = max(pieces, key=len)
    return largest_piece[:1], len(largest_piece)

longest_consecutive_subsequence_1(s)

In [ ]:
check(longest_consecutive_subsequence_1)

### Method 2: Data parallel "preconditioning" approach ###

Transform the input into something, e.g., preprocess or "precondition" it so that the problem becomes easier to solve.

In [ ]:
s = 'bbbaaabaaaa'

### INSERT: DEMO `s` and `t` ###

In [ ]:
def insert_cuts(s, separator='|'):
    assert separator not in s     # Q for you: "but y tho"
    ### YOUR CODE HERE ###
    pass

def longest_consecutive_subsequence_3(s):
    t = insert_cuts(s)
    substrings = t.split('|')
    largest_substring = max(substrings, key=len)
    return largest_substring[-1:], len(largest_substring)

check(longest_consecutive_subsequence_3)

## Summary ##

Here is a quick review of the main ideas in this note:

- **Slices** are well-worth mastering. Empty ranges produced empty lists, which we exploited to write more robust code for corner cases.
- **Functions** encapsulate ... functionality!
- **Data parallelism** is an "all-at-once" mindset, which can be more efficient, easier to read, and easier to parallelize.
- **Preconditioning** is a strategy whereby one transforms an input (or any intermediate object) into another object that simplifies how the next step can be solved.